In [14]:
import pandas as pd
import numpy as np

# 1. Cargar los datos crudos desde la capa Bronze
# Se utiliza header=None porque el archivo tiene problemas de formato 
# que requieren una limpieza manual antes de asignar encabezados.
df_raw = pd.read_csv(
    "../datos/bronze/raw_data_customers.csv",
    header=None
)

# 2. Limpieza de formato
# Algunos campos pueden estar envueltos en comillas adicionales.
df_raw[0] = df_raw[0].str.strip('"')

# 3. Parseo manual de columnas
# Se expande la cadena única separada por comas en columnas independientes.
df = df_raw[0].str.split(",", expand=True)

# 4. Configuración de encabezados
# Tomamos la primera fila (que contiene los nombres de las columnas) y la asignamos.
df.columns = df.iloc[0]   # fila 0 como encabezados

# 5. Limpieza final de estructura
# Eliminamos la fila de encabezados del dataset y reiniciamos el índice.
df = df.drop(index=0).reset_index(drop=True)

# Visualizar el resultado de la carga
df.head()

,customer_id,full_name,email,phone,signup_date,last_purchase_date,monthly_spend,total_shipments,churn_label,home_address
0,C001,Juan Perez,jperez@email.com,555-0101,2023-01-15,12/05/2025,450.50,12,0,Calle Falsa 123
1,C002,Maria Garcia,m.garcia@provider.net,555-0102,2023-02-20,2025-05-10,1200.00,45,0,Carrera 7 # 45-10
2,C003,Carlos Rodriguez,c.rod@work.com,,2023-03-05,03/25/2025,NULL,8,1,Av. Siempre Viva 742
3,C004,Ana Martinez,ana.mtz@mail.com,555-0104,2023-04-12,2025-06-01,890.20,22,0,Clle 100 # 15
4,C001,Juan Perez,jperez@email.com,555-0101,2023-01-15,12/05/2025,450.50,12,0,Calle Falsa 123


Este análisis permite identificar:
- Cantidad de registros y variables
- Tipos de datos incorrectos (fechas como string, numéricos como object)
- Variables candidatas a limpieza o estandarización


In [ ]:
print(df.shape) #Filas y columnas

print(df.columns) #Nombres de las columnas

print(df.info()) #Información de las columnas

(114, 10)
Index(['customer_id', 'full_name', 'email', 'phone', 'signup_date',
       'last_purchase_date', 'monthly_spend', 'total_shipments', 'churn_label',
       'home_address'],
      dtype='object', name=0)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         114 non-null    object
 1   full_name           114 non-null    object
 2   email               114 non-null    object
 3   phone               114 non-null    object
 4   signup_date         114 non-null    object
 5   last_purchase_date  114 non-null    object
 6   monthly_spend       114 non-null    object
 7   total_shipments     114 non-null    object
 8   churn_label         114 non-null    object
 9   home_address        114 non-null    object
dtypes: object(10)
memory usage: 9.0+ KB
None


Este análisis es clave para definir:
- Estrategias de imputación
- Eliminación de variables
- Creación de indicadores de ausencia de información


In [18]:
nulos = df.isnull().sum().to_frame(name="nulos")
nulos["porcentaje"] = (nulos["nulos"] / len(df)) * 100
nulos.sort_values(by="porcentaje", ascending=False)


,nulos,porcentaje
0,,
customer_id,0,0.00
full_name,0,0.00
email,0,0.00
phone,0,0.00
signup_date,0,0.00
last_purchase_date,0,0.00
monthly_spend,0,0.00
total_shipments,0,0.00
churn_label,0,0.00


Aunque el dataset no presenta valores nulos explícitos (NaN),
se identifican valores vacíos y representaciones no estandarizadas
de datos faltantes (ej. strings vacíos, espacios en blanco y valores
como 'NA' o 'NULL').

Desde la perspectiva de Data Governance, estos valores deben
ser tratados como nulos lógicos y normalizados antes del modelado.


In [19]:
posibles_nulos = ["NA", "N/A", "NULL", "null", "None", "none"]

nulos_logicos = (
    df.select_dtypes(include="object")
      .apply(lambda col: col.isin(posibles_nulos))
      .sum()
      .to_frame(name="nulos_logicos")
)

nulos_logicos


,nulos_logicos
0,
customer_id,0
full_name,0
email,0
phone,0
signup_date,0
last_purchase_date,1
monthly_spend,16
total_shipments,1
churn_label,0


In [21]:
posibles_nulos = ["NA", "N/A", "NULL", "null", "None", "none"]

nulos_logicos = (
    df.select_dtypes(include="object")
      .apply(lambda col: col.isin(posibles_nulos))
      .sum()
      .to_frame(name="nulos_logicos")
)

nulos_logicos


,nulos_logicos
0,
customer_id,0
full_name,0
email,0
phone,0
signup_date,0
last_purchase_date,1
monthly_spend,16
total_shipments,1
churn_label,0


Los duplicados completos pueden indicar problemas en la ingesta de datos
o reprocesos operativos.


In [22]:
duplicados_totales = df.duplicated().sum()
duplicados_totales

3

### Análisis de variables categóricas
Este análisis permite:
- Identificar cardinalidad alta
- Detectar posibles identificadores
- Evaluar riesgo de sobreajuste o fuga de información

In [23]:
columnas_categoricas = df.select_dtypes(include="object").columns

df[columnas_categoricas].nunique().sort_values(ascending=False)

0
customer_id           110
full_name             110
email                 110
phone                 110
signup_date            96
home_address           96
monthly_spend          66
last_purchase_date     54
total_shipments        54
churn_label             3
dtype: int64

### Análisis de variables numéricas

En el estado actual del dataset, ninguna columna es reconocida
como numérica por Pandas. Esto indica que variables cuantitativas
se encuentran almacenadas como strings, posiblemente debido a
formatos inconsistentes, separadores decimales o valores vacíos.

Este hallazgo refuerza la necesidad de una etapa formal de
Data Cleaning y normalización antes del modelado.


In [25]:
columnas_numericas = df.select_dtypes(include=[np.number]).columns

if len(columnas_numericas) > 0:
    df[columnas_numericas].describe().T
else:
    print("⚠️ No se detectaron columnas numéricas en el dataset en su estado raw.")


⚠️ No se detectaron columnas numéricas en el dataset en su estado raw.


### Análisis de fechas

Se evalúa:
- Consistencia de formatos
- Necesidad de normalización
- Viabilidad para feature engineering temporal

In [ ]:
columnas_fechas = df.columns[df.columns.str.contains("date|fecha", case=False)]

columnas_fechas

Index(['signup_date', 'last_purchase_date'], dtype='object', name=0)

In [28]:
for col in columnas_fechas:
    print(f"\nColumna: {col}")
    print(df[col].head())



Columna: signup_date
0    2023-01-15
1    2023-02-20
2    2023-03-05
3    2023-04-12
4    2023-01-15
Name: signup_date, dtype: object

Columna: last_purchase_date
0    12/05/2025
1    2025-05-10
2    03/25/2025
3    2025-06-01
4    12/05/2025
Name: last_purchase_date, dtype: object



### Identificación de variables sensibles (PII)
A partir del nombre y contenido de las columnas, se identifican posibles
variables sensibles como:
- Nombres
- Correos electrónicos
- Teléfonos
- Direcciones
- Identificadores personales

Estas variables deberán ser eliminadas o anonimizadas en la fase de Data Governance.


In [26]:
df.columns


Index(['customer_id', 'full_name', 'email', 'phone', 'signup_date',
       'last_purchase_date', 'monthly_spend', 'total_shipments', 'churn_label',
       'home_address'],
      dtype='object', name=0)

### Hallazgos principales

1. El dataset presenta valores nulos significativos en varias columnas.
2. Existen variables con tipos de datos inconsistentes.
3. Se detectan columnas con alta cardinalidad que podrían ser identificadores.
4. Hay riesgo de uso de variables sensibles no aptas para modelado.
5. El dataset requiere un proceso formal de Data Governance antes de cualquier
   modelado predictivo.

### Próximo paso:
Implementar reglas de limpieza, anonimización y validación de calidad de datos.
